# AI 이용 주식 종목 추천

In [1]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

import yfinance as yf
import pandas as pd

In [2]:
import FinanceDataReader as fdr

# 주식 종목 리스트 (예: 삼성전자, 에코프로 등)
stocks = [('005930.KS', '삼성전자'), ('086520.KQ', '에코프로'), ('003490.KS', '대한항공'), ('373220.KS', 'LG엔솔'), 
          ('207940.KS', '삼성바이오로직스'), ('012450.KS', '한화에어로스페이스')] 

# 주식 데이터 다운로드 (yfinance를 사용)
def download_stock_data(ticker, start_date="2024-01-01"):
    stock_data = fdr.DataReader(ticker[0][:6], start_date)  #셀트리온
    return stock_data

# 주식 데이터 수집
stock_data = {}
for stock in stocks:
    stock_data[stock] = download_stock_data(stock)

stock_data

{('005930.KS',
  '삼성전자'):              Open   High    Low  Close    Volume    Change
 Date                                                      
 2024-01-02  78200  79800  78200  79600  17142847  0.014013
 2024-01-03  78500  78800  77000  77000  21753644 -0.032663
 2024-01-04  76100  77300  76100  76600  15324439 -0.005195
 2024-01-05  76700  77100  76400  76600  11304316  0.000000
 2024-01-08  77000  77500  76400  76500  11088724 -0.001305
 ...           ...    ...    ...    ...       ...       ...
 2025-02-14  56000  57300  56000  56000  23979779  0.003584
 2025-02-17  56100  56500  55700  56000  11916027  0.000000
 2025-02-18  56200  57200  55900  56900  22131007  0.016071
 2025-02-19  57400  58900  57300  58700  25395151  0.031634
 2025-02-20  59100  59100  58100  58400  21465778 -0.005111
 
 [276 rows x 6 columns],
 ('086520.KQ',
  '에코프로'):               Open    High     Low   Close   Volume    Change
 Date                                                         
 2024-01-02  1295

In [3]:
# 상승 가능성 판단을 위한 간단한 분석 (예: 이동평균 분석)
def calculate_moving_averages(stock_data):
    stock_data['SMA20'] = stock_data['Close'].rolling(20).mean()
    stock_data['SMA50'] = stock_data['Close'].rolling(50).mean()
    # 현재 SMA20 > SMA50인 경우 상승 가능성 있다고 판단
    # return stock_data['SMA20'].iloc[-1] > stock_data['SMA50'].iloc[-1] 
    return True

# 상승 가능성 높은 종목 리스트
potential_rising_stocks = []
for stock, data in stock_data.items():
    if calculate_moving_averages(data):
        potential_rising_stocks.append(stock)

potential_rising_stocks

[('005930.KS', '삼성전자'),
 ('086520.KQ', '에코프로'),
 ('003490.KS', '대한항공'),
 ('373220.KS', 'LG엔솔'),
 ('207940.KS', '삼성바이오로직스'),
 ('012450.KS', '한화에어로스페이스')]

In [4]:
import yfinance as yf

# financials 딕셔너리 생성
financials = {}

# 상승 가능성이 있는 주식들의 티커 리스트(potential_rising_stocks)에서 각 주식의 재무 상태를 가져옵니다.
for ticker in potential_rising_stocks:
    # 각 티커에 해당하는 주식 데이터를 yfinance에서 불러옵니다.
    stock = yf.Ticker(ticker[0])
    
    # 해당 주식의 재무 정보를 저장할 딕셔너리 구조를 생성합니다.
    financials[ticker] = {}
    
    # 손익계산서(Income Statement) 데이터를 가져와서 financials 딕셔너리에 저장합니다.
    financials[ticker]['income statement'] = stock.financials
    
    # 대차대조표(Balance Sheet) 데이터를 가져와서 financials 딕셔너리에 저장합니다.
    financials[ticker]['balance sheet'] = stock.balance_sheet
    
    # 현금흐름표(Cash Flow Statement) 데이터를 가져와서 financials 딕셔너리에 저장합니다.
    financials[ticker]['cash flow'] = stock.cashflow

# 저장된 financials 딕셔너리의 키(티커 목록)를 출력합니다.
print(financials.keys())

# 전체 재무 데이터를 출력합니다.
financials

dict_keys([('005930.KS', '삼성전자'), ('086520.KQ', '에코프로'), ('003490.KS', '대한항공'), ('373220.KS', 'LG엔솔'), ('207940.KS', '삼성바이오로직스'), ('012450.KS', '한화에어로스페이스')])


{('005930.KS',
  '삼성전자'): {'income statement':                                                             2024-12-31  \
  Tax Effect Of Unusual Items                           -7643351884.3752   
  Tax Rate For Calcs                                            0.082025   
  Normalized EBITDA                                     81157657000000.0   
  Total Unusual Items                                     -93183000000.0   
  Total Unusual Items Excluding Goodwill                  -93183000000.0   
  Net Income From Continuing Operation Net Minori...    33621363000000.0   
  Reconciled Depreciation                               42630822000000.0   
  Reconciled Cost Of Revenue                           186562268000000.0   
  EBITDA                                                81064474000000.0   
  EBIT                                                  38433652000000.0   
  Net Interest Income                                    3915210000000.0   
  Interest Expense                         

In [5]:
# OpenAI API로 상승 가능성 높은 종목에 대한 분석 요청
# 분석할 주식 리스트와 재무 상태를 기반으로 프롬프트를 작성합니다.
prompt = f"다음 주식들은 기술적 지표에 기반하여 상승 가능성이 있습니다: {', '.join(str(potential_rising_stocks))}. " + \
         "각 주식의 재무 상태는 다음과 같습니다:\n"

# 상승 가능성이 높은 종목의 재무 상태를 프롬프트에 추가합니다.
for ticker in potential_rising_stocks:
    prompt += f"\n{ticker}의 재무 상태:\n{financials[ticker]}\n"

# 프롬프트에 주식 추천 요청을 추가합니다.
prompt += "이 주식들 중에 어떤 주식이 가장 투자에 유망한지 두 종목을 추천해 주고 그 이유를 설명해 주세요. 한글로 답해주세요."

# OpenAI API로 질문을 보내고, 분석 결과를 받습니다.
response = client.chat.completions.create(
    model=Model,  # 사용할 모델을 지정합니다.
    messages=[
        {"role": "system", "content": "You are a helpful investment professional."},  # 시스템 역할 정의
        {"role": "user", "content": prompt},  # 사용자의 질문을 프롬프트로 전달
    ]
)

# OpenAI로부터 받은 응답을 출력합니다.
print(response.choices[0].message.content)

투자에 유망한 두 종목으로 **삼성전자 (005930.KS)**와 **대한항공 (003490.KS)**를 추천합니다.

### 1. 삼성전자 (005930.KS)
- **재무 상태**: 삼성전자는 강력한 수익성과 안정적인 재무 구조를 보이고 있습니다. 최근 순이익과 EBITDA가 긍정적인 성장세를 보이며, 특히, 지속적인 연구개발 투자와 함께 미래 성장 동력 확보를 위해 노력하고 있습니다. 다수의 기초 데이터에서 안정적인 캐시 포지션과 매출 성장을 확인할 수 있습니다.
- **산업 위치**: 삼성전자는 반도체 및 전자 기술 분야에서 세계 최고의 업체 중 하나로, 글로벌 경제가 회복됨에 따라 수요가 증가할 것으로 기대됩니다. 반도체 공급망의 회복과 함께 디스플레이 및 모바일 분야에서도 시장 점유율 확대가 가능할 것으로 보입니다.
- **기술적 지표**: 기술적 분석에서도 최근 주가가 상승 추세를 보이고 있으며, 저가 매수 구간에 위치해 있는 것으로 분석됩니다.

### 2. 대한항공 (003490.KS)
- **재무 상태**: 항공 산업이 COVID-19 회복세를 보이며, 대한항공은 이익 개선을 목표로 하고 있습니다. 최근 순이익이 긍정적으로 나타나고 있으며, 여객 수요 증가 및 화물 운송 호조가 예상됩니다. 이로 인해 수익성이 개선될 것으로 기대됩니다.
- **산업 회복**: 최근 글로벌 여행 수요가 증가함에 따라 항공사들의 실적이 회복되고 있으며, 대한항공 역시 국제선 공항노선의 확대 및 동남아시아 및 북미 노선에서의 수익 증가가 예상됩니다.
- **기술적 지표**: 대한항공의 주가는 최근 긍정적인 상승세를 보이고 있으며, RSA 및 MACD와 같은 기술적 지표도 긍정적인 신호를 나타내고 있습니다.

### 결론
두 종목 모두 기술적 지표와 재무 상태가 양호하며, 각각의 산업에서 회복세를 보이고 있습니다. 삼성전자는 안정적인 성장을 주도할 가능성이 높고, 대한항공은 여행 수요 증가로 인한 수익성 회복이 기대됩니다. 이러한 이유로 이 두 종목을 추천합니다.
